<a href="https://colab.research.google.com/github/urmilapol/urmilapolprojects/blob/master/Copy_of_beautifulsoup1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
r = requests.get('https://www.renthop.com/boston-ma/apartments-for-rent')
r.content

b'<!doctype html>\n<html lang="en">\n<head>\n<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />\n<meta http-equiv="Content-Language" content="en" />\n<title>Apartments for Rent in Boston, MA, No Fee Rentals | RentHop</title>\n<meta name="description" content="Newly listed Boston, MA apartments for rent. Smooth work commute, popular bars and nightlife, nearby restaurants and grocery stores, and safety. Find your perfect home in Boston, MA." />\n<meta name="author" content="RentHop" />\n<meta name="Copyright" content="Copyright (c) 2009 - 2021 RentHop.com" />\n<meta property="fb:page_id" content="124300320712" />\n<meta property="fb:app_id" content="294321126236" />\n<meta name="og:image" content="https://www.renthop.com/images/renthop_icon_small.png" />\n<link rel="image_src" href="https://www.renthop.com/images/renthop_icon_small.png" />\n<meta name="og:title" content="Apartments for Rent in Boston, MA, No Fee Rentals | RentHop" />\n<meta name="og:description" conten

In [ ]:
# Creating an instance of BeautifulSoup
soup = BeautifulSoup(r.content, "html5lib")
listing_divs = soup.select('div[class*=search-info]')
print(listing_divs)

[<div class="search-info pl-md-4">
<div>
<div class="float-right font-size-9" style="padding-top: 2px;">
<span class="font-gray-2 d-none d-sm-inline-block">1 hour ago</span>
<span class="d-none d-sm-inline-block" style="color: #dddddd;"> | </span> <span class="d-none d-sm-inline-block font-gray-2">Score:</span>
<span class="d-none d-sm-inline-block b font-blue" id="listing-61045052-hopscore" style="">98.0</span>
</div>
<a class="font-size-11 listing-title-link b" href="https://www.renthop.com/listings/195-hancock-street/na/61045052" id="listing-61045052-title">195 Hancock Street</a>
<div class="font-size-9 overflow-ellipsis" id="listing-61045052-neighborhoods" style="line-height: 130%;">
Uphams Corner - Jones Hill, Dorchester, Boston
</div>
</div>
<div style="margin-top: 8px;">
<div id="listing-61045052-info">
<div class="d-inline-block align-bottom" id="listing-61045052-price">
<span class="font-size-13 b">$1,250</span>
</div>
<div class="d-inline-block align-middle">
<div class="font

In [ ]:
# Retrieving data from one record
url = listing_divs[0].select('a[id*=title]')[0]['href']
address = listing_divs[0].select('a[id*=title]')[0].string
neighborhood = listing_divs[0].select('div[id*=hood]')[0].string.replace('\n','')
print(url)
print(address)
print(neighborhood)

https://www.renthop.com/listings/195-hancock-street/na/61045052
195 Hancock Street
Uphams Corner - Jones Hill, Dorchester, Boston


In [ ]:
url_prefix = "https://www.renthop.com/search/boston-ma?min_price=0&max_price=50000&q=&sort=hopscore&search=0&page="
page_number = 0
for url in range(4):
    target_page = url_prefix + str(page_number)
    print(target_page + '\n')
    page_number += 1


https://www.renthop.com/search/boston-ma?min_price=0&max_price=50000&q=&sort=hopscore&search=0&page=0

https://www.renthop.com/search/boston-ma?min_price=0&max_price=50000&q=&sort=hopscore&search=0&page=1

https://www.renthop.com/search/boston-ma?min_price=0&max_price=50000&q=&sort=hopscore&search=0&page=2

https://www.renthop.com/search/boston-ma?min_price=0&max_price=50000&q=&sort=hopscore&search=0&page=3



In [ ]:
# Creating a function to retrieve data from a list of <div>s
def retrieve_data(listing_divs):    
    listing_list = []
    for index in range(len(listing_divs)):
        each_listing = []
        current_listing = listing_divs[index]
        url = current_listing.select('a[id*=title]')[0]['href']
        address = current_listing.select('a[id*=title]')[0].string
        neighborhood = current_listing.select('div[id*=hood]')[0].string.replace('\n','')
        each_listing.append(url)
        each_listing.append(address)
        each_listing.append(neighborhood)
        listing_specs = current_listing.select('table[id*=info] tr') 
        for spec in listing_specs:
            try:
                each_listing.extend(spec.text.strip().replace(' ','_').split())
            except:
                each_listing.extend(np.nan)
        listing_list.append(each_listing)
    return listing_list
# Looping and getting data from 350 pages (part of the result of searching)
url_prefix = "https://www.renthop.com/search/boston-ma?min_price=0&max_price=50000&q=&sort=hopscore&search=0&page="
page_number = 1
all_pages_parsed = []
pages = 350
for url in range(pages):
    target_page = url_prefix + str(page_number)
    page_number += 1
    r = requests.get(target_page)
    
    # Getting a BeautifulSoup instance to be able to retrieve data
    soup = BeautifulSoup(r.content, "html5lib")
    listing_divs = soup.select('div[class*=search-info]')
    
    one_page_parsed = retrieve_data(listing_divs)
    all_pages_parsed.extend(one_page_parsed)

In [ ]:
df = pd.DataFrame(all_pages_parsed, columns=['url','address','neighborhood'])
df.head()

,url,address,neighborhood
0,https://www.renthop.com/listings/195-hancock-s...,195 Hancock Street,"Uphams Corner - Jones Hill, Dorchester, Boston"
1,https://www.renthop.com/listings/114-boston-st...,114 Boston Street,"Columbia Point, Dorchester, Boston"
2,https://www.renthop.com/listings/55-traveler-s...,"55 Traveler St., Apt B","Shawmut, South End, Boston"
3,https://www.renthop.com/listings/660-washingto...,"660 Washington St., Apt 2 BEDR...","Chinatown - Leather District, Boston"
4,https://www.renthop.com/listings/50-pratt-st/n...,50 Pratt St,"Allston, Boston"


In [ ]:
# Writing a comma-separated values (CSV) file
df.to_csv('apartments_leasing.csv', index=False)